In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the data from csv file
data = pd.read_csv("C:\\Users\\exx\\Downloads\\Rosemount_LeafOn_SfM_Canopies_PointVertexCoords.csv")

In [10]:
# list columns in data
print(data.columns)

Index(['treeID', 'diameter', 'area', 'volume', 'row', 'tree', 'xmin', 'ymin',
       'xmax', 'ymax', 'width', 'length', 'POINT_X', 'POINT_Y'],
      dtype='object')


In [11]:
# width = xmax - xmin (in UTM)
# length = ymax - ymin (in UTM)
new_data = data.copy()

# only keep the first row of each unique treeID
new_data = new_data.drop_duplicates(subset='treeID')

# drop POINT_X and POINT_Y columns
new_data = new_data.drop(columns=['POINT_X', 'POINT_Y'])

# print first 5 rows
print(new_data.head())


       treeID  diameter      area    volume  row  tree           xmin  \
0           4  2.144426  3.660047  4.315648    2     1  492090.907855   
2709        5  1.414493  1.647882  0.565725    2    11  492118.886217   
4996        6  2.787853  6.299513  5.092399    2     4  492098.282423   
11641       7  1.218108  1.193735  0.474788    2     2  492094.017673   
13434       8  1.369170  1.534295  0.608245    2     6  492104.917035   

               ymin           xmax          ymax     width   length  
0      4.952560e+06  492093.227719  4.952563e+06  2.319864  2.42486  
2709   4.952561e+06  492120.501123  4.952563e+06  1.614906  1.57991  
4996   4.952560e+06  492102.322187  4.952563e+06  4.039764  2.81483  
11641  4.952561e+06  492095.487587  4.952562e+06  1.469914  1.35492  
13434  4.952561e+06  492106.631934  4.952562e+06  1.714899  1.61991  


In [12]:
# calculate diagonal width, length, and diameter for each tree
new_data.loc[:, 'diagonal_width'] = np.nan
new_data.loc[:, 'diagonal_length'] = np.nan
new_data.loc[:, 'diameter'] = np.nan

for treeID in new_data['treeID']:
    tree = data[data['treeID'] == treeID].copy()
    tree.loc[:, 'x_ymin'] = min(tree.loc[tree['POINT_Y'] == tree['POINT_Y'].min(), 'POINT_X'])
    tree.loc[:, 'x_ymax'] = max(tree.loc[tree['POINT_Y'] == tree['POINT_Y'].max(), 'POINT_X'])
    tree.loc[:, 'y_xmin'] = min(tree.loc[tree['POINT_X'] == tree['POINT_X'].min(), 'POINT_Y'])
    tree.loc[:, 'y_xmax'] = max(tree.loc[tree['POINT_X'] == tree['POINT_X'].max(), 'POINT_Y'])
    tree.loc[:, 'diagonal_width'] = np.sqrt(((tree['xmax'] - tree['xmin'])**2 + (tree['y_xmax'] - tree['y_xmin'])**2))
    tree.loc[:, 'diagonal_length'] = np.sqrt(((tree['ymax'] - tree['ymin'])**2 + (tree['x_ymax'] - tree['x_ymin'])**2))

    # find diameter (this takes a while to execute)
    max_distance = 0
    for i in range(len(tree)):
        for j in range(i+1, len(tree)):
            # diameter is longest euclidean distance between any two points in polygon
            distance = np.sqrt((tree.iloc[i]['POINT_X'] - tree.iloc[j]['POINT_X'])**2 + (tree.iloc[i]['POINT_Y'] - tree.iloc[j]['POINT_Y'])**2)
            if distance > max_distance:
                max_distance = distance

    new_data.loc[new_data['treeID'] == treeID, 'diameter'] = max_distance
    new_data.loc[new_data['treeID'] == treeID, 'diagonal_width'] = tree['diagonal_width'].values[0]
    new_data.loc[new_data['treeID'] == treeID, 'diagonal_length'] = tree['diagonal_length'].values[0]
    new_data.loc[new_data['treeID'] == treeID, 'x_ymin'] = float(tree['x_ymin'].values[0])
    new_data.loc[new_data['treeID'] == treeID, 'x_ymax'] = float(tree['x_ymax'].values[0])
    new_data.loc[new_data['treeID'] == treeID, 'y_xmin'] = float(tree['y_xmin'].values[0])
    new_data.loc[new_data['treeID'] == treeID, 'y_xmax'] = float(tree['y_xmax'].values[0])

In [13]:
print(new_data.head())

       treeID  diameter      area    volume  row  tree           xmin  \
0           4  2.471843  3.660047  4.315648    2     1  492090.907855   
2709        5  1.690618  1.647882  0.565725    2    11  492118.886217   
4996        6  4.138676  6.299513  5.092399    2     4  492098.282423   
11641       7  1.675114  1.193735  0.474788    2     2  492094.017673   
13434       8  1.729754  1.534295  0.608245    2     6  492104.917035   

               ymin           xmax          ymax     width   length  \
0      4.952560e+06  492093.227719  4.952563e+06  2.319864  2.42486   
2709   4.952561e+06  492120.501123  4.952563e+06  1.614906  1.57991   
4996   4.952560e+06  492102.322187  4.952563e+06  4.039764  2.81483   
11641  4.952561e+06  492095.487587  4.952562e+06  1.469914  1.35492   
13434  4.952561e+06  492106.631934  4.952562e+06  1.714899  1.61991   

       diagonal_width  diagonal_length         x_ymin         x_ymax  \
0            2.320774         2.425731  492091.737806  492091.

In [14]:
# save df to csv
new_data.to_csv("C:\\Users\\exx\\Downloads\\Rosemount_SfM_Canopies_Euclidean_HW.csv", index=False)